In [36]:
import pandas as pd
from collections import deque, OrderedDict
from datetime import timedelta, datetime, date, time
import os, pytz, calendar, urllib2, json, re, requests, pprint
import plotly.plotly as py
import plotly.graph_objs as go

## Configuration

In [22]:
#chemical names and associated health threshold
healthlimits = OrderedDict([("Benzene",          1), 
                            ("Black_Carbon",     5), 
                            ("Ethylbenzene",    60), 
                            ("Hydrogen_Sulfide", 8),
                            ("Sulfur_Dioxide",  75),
                            ("Toluene",         70),
                            ("Xylene",          50)])

#investigation time frame
#April 2017
startDay = datetime(2017,4,1)
duration = 30 #days

## Helper Functions

### Time helper functions

In [47]:
#returns start and end timestamps of provided start date and duration in number of days
def getEpochTimeBounds(d, duration):
    dt = assignPacificTimeZone(datetime(d.year,d.month,d.day))
    start = calendar.timegm(dt.utctimetuple())
    end = calendar.timegm((dt + timedelta(days=duration)).utctimetuple())
    return {'start' : start, 'end': end}

#attach DST aware timezone offset
#Note: does not convert time
def assignPacificTimeZone(dt):
    pacific = pytz.timezone("US/Pacific")
    dt = pacific.localize(dt)
    return dt

#convert either a unix timestamp or a datetime with tzinfo to a datetime in Pacific time
def convertToPacific(time):
    if not isinstance(time,datetime):
        time = datetime.fromtimestamp(time,tz=pytz.utc)
    pacific = pytz.timezone("US/Pacific")
    inPacific = time.astimezone(pacific)
    return inPacific

#returns a date range generator to be used to capture specific days:
# Ex:
#for single_date in daterange(startDay, duration):
#    print single_date.strftime("%Y-%m-%d")
def daterange(start_date, duration):
    for n in range(duration):
        yield start_date + timedelta(n)
        
#generator for 24 hour times
#Ex:
#for single_hour in twentyfourhourrange():
#     print single_hour.strftime("%H:%M:%S")
def twentyfourhourrange():
    for n in range(24):
        yield (datetime.combine(date.today(), time(0)) + timedelta(hours=(1*n))).time()

### Fenceline ESDR data helper functions

In [30]:
#converts a fenceline feed name to a location
def parsefeedcommunity(feedname):
    return str(feedname.split('fenceline_org')[0].strip().split('Fence')[0].strip())

#returns if an ESDR chemical name relates to a chemical name
def parsechemicalname(esdrchemicalname, chemicalname):
    return esdrchemicalname.find(chemicalname) > -1

#returns a copy of the interested data columns using chemical_map (generated below)
def getchemicaldata(monitor, chemical, fenceline_data, chemical_map):
    col_names = chemical_map[monitor][chemical]
    return fenceline_data[monitor][col_names].copy().replace("[^0-9]+",0,regex=True)

#Ex: getchemicaldata('Point Richmond Refinery',"Hydrogen_Sulfide",fenceline_data, chemicalmap)

## ESDR Functions

In [33]:
#returns ESDR data as a json
def makeESDRrequest(urlsuffix):
    url = "https://esdr.cmucreatelab.org/api/v1/feeds/%s" % urlsuffix
    return json.loads(urllib2.urlopen(url).read())['data']

#returns the feed information for a given feedID
def loadfeed(feedID):
    return makeESDRrequest(feedID)

#returns all of the channels for a given feed data (use with loadfeed)
def getchannels(feedData):
    return [str(channel) for channel in feedData['channelBounds']['channels'].keys()]

#returns info on all of the fenceline feeds
def getfencelinefeedinfo():
    return makeESDRrequest('?fields=id,name,latitude,longitude&whereOr=productId=36&orderBy=+id')['rows']    

In [34]:
#feedID: integer
#esdrChannels: a list of channel names as strings
#timeOptions: a dictionary as {bounds: {start: epochInt, end: epochInt}}, or {day: datetime, duration: int}
def makedataframefromESDR(feedID, 
                          timeOptions = {}):
    if timeOptions.get('bounds') == None:
        bounds = getEpochTimeBounds(
            timeOptions.get('day') or datetime.now()-timedelta(1), 
            timeOptions.get('duration') or 1)
    else:
        bounds = timeOptions.get('bounds')
        
    esdrChannels = getchannels(loadfeed(feedID))
    
    try:
        r = makeESDRrequest("%s/channels/%s/export?from=%s&to=%s&format=json" % (feedID, ','.join(esdrChannels), bounds['start'], bounds['end']))
        print "loaded " + str(len(r)) + " data points for feed " + str(feedID) + ", channels: " + '|'.join(esdrChannels) + ", time " + str(bounds['start'])
    except:
        print "error loading data from ESDR: feed " + str(feedID) + ", channel " + '|'.join(esdrChannels) + ", time " + str(bounds['start'])
    cols = ['Time']
    cols.extend(esdrChannels)
    df = pd.DataFrame(r,columns=cols)
    df['Time'] = pd.to_datetime(df['Time'],unit='s').dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
    return df.set_index(['Time'])

## Import Data

In [7]:
#time frame: 1 month of April, 2017
timeframe = {'day':startDay, 'duration':duration}

#retrieves fenceline feed information (location, name, feed id)
fenceline_feeds = getfencelinefeedinfo()

#loads all the data from every channel and every fenceline feed, separated by feed
fenceline_data = {str(parsefeedcommunity(feed['name'])):makedataframefromESDR(feed['id'],timeframe) for feed in fenceline_feeds}

loaded 25287 data points for feed 4901, channels: FTIR_System_Status|UV_Signal_Strength|FTIR_Ethanol|FTIR_Mercaptan|FTIR_System_Manufacturer|FTIR_Ammonia|UV_Ozone|FTIR_Methane|FTIR_Carbon_Monoxide|UV_Carbon_Disulfide|FTIR_Nitrous_Oxide|TDL_Signal_Strength|FTIR_Carbonyl_Sulfide|UV_Benzene|FTIR_Total_Hydrocarbons|FTIR_1_3_Butadiene|FTIR_Ethylene|TDL_Hydrogen_Sulfide|UV_Xylene|UV_Toluene|UV_Sulfur_Dioxide|UV_System_Status|FTIR_MTBE, time 1491030000
loaded 24448 data points for feed 4902, channels: FTIR_System_Status|UV_Signal_Strength|FTIR_Ethanol|FTIR_Mercaptan|FTIR_System_Manufacturer|FTIR_Ammonia|UV_Ozone|FTIR_Methane|FTIR_Carbon_Monoxide|UV_Carbon_Disulfide|FTIR_Nitrous_Oxide|TDL_Signal_Strength|FTIR_Carbonyl_Sulfide|UV_Benzene|FTIR_Total_Hydrocarbons|FTIR_1_3_Butadiene|FTIR_Ethylene|TDL_Hydrogen_Sulfide|UV_Xylene|UV_Toluene|UV_Sulfur_Dioxide|UV_System_Status|FTIR_MTBE, time 1491030000
loaded 16910 data points for feed 4903, channels: Wind_Speed_MPH|Wind_Direction|OGD_System_Status|Hu

### Build Chemical to Column Mapping

In [8]:
fenceline_columns = {monitor: [x for x in fenceline_data[monitor].columns]
                     for monitor in fenceline_data}

chemicalmap = {monitor: {chemical:[x for x in fenceline_data[monitor].columns if parsechemicalname(x, chemical)] 
                     for chemical in healthlimits}
               for monitor in fenceline_data}

## Data Analysis Algorithms

In [67]:
def countsabovehealthlimit(df, chemical):
    countsabovehealthlimit = [];
    healthlimit = healthlimits[chemical]
    for column in df:
        countsabovehealthlimit.append(df[df[column] > healthlimit].count(numeric_only=True).tolist()[0])
    return countsabovehealthlimit

## Analyze Data Hourly

In [90]:
#Retrieves number of 
daily_data={}
for monitor in fenceline_data.keys():
    daily_data[monitor]={};
    for chemical in healthlimits:
        daily_data[monitor][chemical]={}
        df = getchemicaldata(monitor, chemical, fenceline_data, chemicalmap)
        for single_date in daterange(startDay, duration):
            daily_data[monitor][chemical][single_date]={}
            healthdaycount = 0
            presencedaycount = 0
            one_day_data = df[single_date.strftime("%Y-%m-%d")]
            for single_hour in twentyfourhourrange():
                next_hour = datetime.combine(date.today(), single_hour) + timedelta(hours=1)                
                one_hour_data = one_day_data.between_time(single_hour.strftime("%H:%M:%S"), next_hour.strftime("%H:%M:%S"))
                healthdaycount += 1 if sum(countsabovehealthlimit(one_hour_data, chemical) else 0
            daily_data[monitor][chemical][single_date]['hoursabovehealthlimit']=healthdaycount

2017-04-01 00:00:00 13
2017-04-02 00:00:00 7
2017-04-03 00:00:00 2
2017-04-04 00:00:00 0
2017-04-05 00:00:00 0
2017-04-06 00:00:00 0
2017-04-07 00:00:00 0
2017-04-08 00:00:00 0
2017-04-09 00:00:00 0
2017-04-10 00:00:00 0
2017-04-11 00:00:00 0
2017-04-12 00:00:00 0
2017-04-13 00:00:00 0
2017-04-14 00:00:00 0
2017-04-15 00:00:00 0
2017-04-16 00:00:00 0
2017-04-17 00:00:00 0
2017-04-18 00:00:00 0
2017-04-19 00:00:00 0
2017-04-20 00:00:00 10
2017-04-21 00:00:00 11
2017-04-22 00:00:00 0
2017-04-23 00:00:00 0
2017-04-24 00:00:00 0
2017-04-25 00:00:00 0
2017-04-26 00:00:00 0
2017-04-27 00:00:00 6
2017-04-28 00:00:00 12
2017-04-29 00:00:00 20
2017-04-30 00:00:00 14
2017-04-01 00:00:00 0
2017-04-02 00:00:00 0
2017-04-03 00:00:00 0
2017-04-04 00:00:00 0
2017-04-05 00:00:00 0
2017-04-06 00:00:00 0
2017-04-07 00:00:00 0
2017-04-08 00:00:00 0
2017-04-09 00:00:00 0
2017-04-10 00:00:00 0
2017-04-11 00:00:00 0
2017-04-12 00:00:00 0
2017-04-13 00:00:00 0
2017-04-14 00:00:00 0
2017-04-15 00:00:00 0
2017

In [82]:
daily_data['North Richmond Refinery']['Hydrogen_Sulfide']

{datetime.datetime(2017, 4, 1, 0, 0): 0,
 datetime.datetime(2017, 4, 2, 0, 0): 0,
 datetime.datetime(2017, 4, 3, 0, 0): 0,
 datetime.datetime(2017, 4, 4, 0, 0): 0,
 datetime.datetime(2017, 4, 5, 0, 0): 0,
 datetime.datetime(2017, 4, 6, 0, 0): 0,
 datetime.datetime(2017, 4, 7, 0, 0): 0,
 datetime.datetime(2017, 4, 8, 0, 0): 0,
 datetime.datetime(2017, 4, 9, 0, 0): 0,
 datetime.datetime(2017, 4, 10, 0, 0): 0,
 datetime.datetime(2017, 4, 11, 0, 0): 0,
 datetime.datetime(2017, 4, 12, 0, 0): 0,
 datetime.datetime(2017, 4, 13, 0, 0): 0,
 datetime.datetime(2017, 4, 14, 0, 0): 0,
 datetime.datetime(2017, 4, 15, 0, 0): 0,
 datetime.datetime(2017, 4, 16, 0, 0): 0,
 datetime.datetime(2017, 4, 17, 0, 0): 0,
 datetime.datetime(2017, 4, 18, 0, 0): 0,
 datetime.datetime(2017, 4, 19, 0, 0): 0,
 datetime.datetime(2017, 4, 20, 0, 0): 0,
 datetime.datetime(2017, 4, 21, 0, 0): 0,
 datetime.datetime(2017, 4, 22, 0, 0): 0,
 datetime.datetime(2017, 4, 23, 0, 0): 0,
 datetime.datetime(2017, 4, 24, 0, 0): 0,
 

In [10]:
getchemicaldata('Point Richmond Refinery',"Hydrogen_Sulfide",fenceline_data, chemicalmap)['2017-04-01':'2017-04-02']

,Hydrogen_Sulfide
Time,
2017-04-01 00:00:00-07:00,0
2017-04-01 00:01:00-07:00,0
2017-04-01 00:02:00-07:00,0
2017-04-01 00:03:00-07:00,0
2017-04-01 00:04:00-07:00,0
2017-04-01 00:05:00-07:00,0
2017-04-01 00:06:00-07:00,0
2017-04-01 00:07:00-07:00,0
2017-04-01 00:08:00-07:00,0
